# ` Seqential Chains `

### - Installing Libraries

In [1]:
!pip install google-cloud-aiplatform

In [2]:
!pip install "shapely<2.0.0"

In [3]:
!pip install google-auth langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 4.0 MB/s eta 0:00:00


### - Configure Plam

In [4]:
credential_path = '/content/drive/MyDrive/data/lang-chain-388911-879e0a870aa4.json'

In [5]:
!export GOOGLE_APPLICATION_CREDENTIALS=credential_path

In [6]:
from google.colab import auth as google_auth
google_auth.authenticate_user()

### - Importing Libraries

In [7]:
import vertexai
from vertexai.preview.language_models import ChatModel, InputOutputTextPair
from langchain.llms import VertexAI
from langchain import PromptTemplate, LLMChain
from langchain.chat_models import ChatVertexAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    AIMessagePromptTemplate
)
from langchain.schema import SystemMessage,AIMessage,HumanMessage
from langchain import FewShotPromptTemplate
from langchain import ConversationChain,LLMChain, PromptTemplate
from langchain.chains import SequentialChain
from langchain.memory import SimpleMemory
from langchain.memory import ConversationBufferMemory, ConversationBufferWindowMemory


### - Vertex Example

In [8]:
vertexai.init(project="lang-chain-388911", location="us-central1")
chat_model = ChatModel.from_pretrained("chat-bison@001")
parameters = {
    "temperature": 0.2,
    "max_output_tokens": 256,
    "top_p": 0.8,
    "top_k": 40
}
chat = chat_model.start_chat()
response = chat.send_message("""Hi""", **parameters)
print(f"Response from Model: {response.text}")

Response from Model: Hello, how can I help you today?


### - Build Sequential Chain

#### Introduction Chain

In [187]:
llm = VertexAI(temperature=.2,top_k =40,max_output_tokens=128 )
intro_template = """As a product manager, you are responsible for gathering requirements for a mobile app. In order to create a clear  and comprehensive
 IEEE SRS document,
 it's important to ask your client the right questions and gather all necessary information. Your goal is to extract the following information:


frist Introduce your self as your name Pickey
after introducing your self in the same message ask him that
a question to get the name of app
after getting the name ask him about main goal of the app
then main service that app provide to use it in the scope
 and after that ask him about target audience
of  app after ask about target audience ask him
about limits of audience like age and demography and geography

don't generate introduction before asking client all questions
and if user asked you to explain or please provide him a description about what you mean
before generate introduction ask him if he want to add any more
data about target users or main goal
to generate very deatiled Introduction of SRS
That is Your Introduction
1. Introduction
- Purpose:
- Scope:
- Intended Audience:


Current conversation:
{history}
Human: {input}
AI:
"""
intro_prompt_template= PromptTemplate(input_variables=['input','history',], template=intro_template)
introduction_chain = ConversationChain(
    llm=llm, verbose=False, memory=ConversationBufferWindowMemory(), prompt= intro_prompt_template,

)

#### Chat

In [58]:
introduction_chain.predict(input = "HI")

"Hi, I'm Pickey, a virtual assistant that helps product managers gather requirements for mobile apps. Can I help you with that?"

In [ ]:
introduction_chain.run(input = "Yes ")

'What is the name of your app?'

In [ ]:
introduction_chain.run(input = "Fit Shop")

'What is the main goal of your app?'

In [ ]:
introduction_chain.predict(input = "I have a small shop and i want to make my customers updated with products and orderde eaasily")

'What kind of services will your app provide?'

In [ ]:
introduction_chain.predict(input = "can you explain more")

'Sure, let me explain what I mean. The scope of your app is the set of features that it will provide. For example, your app might allow customers to browse products, place orders, and track their orders.'

In [ ]:
introduction_chain.predict(input = "yes  iwant my app to have those features ")

'Great, so the scope of your app is to provide a platform for your customers to order products from your shop, including browsing products, adding them to their cart, checking out, and receiving push notifications about new products or sales. Who is the target audience for your app?'

In [ ]:
introduction_chain.predict(input = 'any one want is to buy')

'Sure, so the target audience for your app is anyone who wants to buy products from your shop. Does that sound right?'

In [ ]:
introduction_chain.predict(input = 'any one want is to buy')

'That is Your Introduction\n1. Introduction\n- Purpose: To provide a platform for customers to order products from the Fit Shop.\n- Scope: The app will allow customers to browse products, add them to their cart, checkout, and receive push notifications about new products or sales.\n- Intended Audience: Anyone who wants to buy products from the Fit Shop.\n\nDo you want to add any more information about your target users or main goal?'

In [ ]:
introduction_chain.predict(input = 'no thanks')

'Great, thank you for your time. I will use this information to create a clear and comprehensive SRS document for your app.'

In [ ]:
introduction_chain.memory.chat_memory.messages[-4:]

[HumanMessage(content='yes', additional_kwargs={}, example=False),
 AIMessage(content='That is Your Introduction\n1. Introduction\n- Purpose: To provide a platform for customers to order products from the Fit Shop.\n- Scope: The app will allow customers to browse products, add them to their cart, checkout, and receive push notifications about new products or sales.\n- Intended Audience: Anyone who wants to buy products from the Fit Shop.\n\nDo you want to add any more information about your target users or main goal?', additional_kwargs={}, example=False),
 HumanMessage(content='no thanks', additional_kwargs={}, example=False),
 AIMessage(content='Great, thank you for your time. I will use this information to create a clear and comprehensive SRS document for your app.', additional_kwargs={}, example=False)]

### Introdcution Model

In [153]:
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, validator
from typing import List
# Here's another example, but with a compound typed field.
class Introduction(BaseModel):
    app_name: str = Field(description="app name of mobile application")
    purpose: str = Field(description="purpose of mobile application")
    scope: str = Field(description="scope of mobile app")
    target_user: str = Field(description="target user of mobile app")



introduction_query = introduction_chain.memory.chat_memory.messages[-4:]

parser = PydanticOutputParser(pydantic_object=Introduction)

prompt = PromptTemplate(
    template="get the {query} and extract {information} and generate more tokens",
    input_variables=["query"],
    partial_variables={"information": parser.get_format_instructions()},
)

_input = prompt.format_prompt(query=introduction_query)

output = llm(_input.to_string())

introduction = parser.parse(output)
print(introduction)

app_name='Shopping App' purpose='To allow users to purchase products' scope='The app will allow users to browse products, add products to their cart, and checkout' target_user='Anyone over the age of 18 who lives in Egypt'


In [ ]:
introduction.app_name

'Fit Shop'

#### Overall Description Chain

In [198]:
llm = VertexAI(temperature=.2,top_k =40,max_output_tokens=128 )
template = f"""As a product manager, you are responsible for gathering requirements for a mobile app. In order to create a clear  and comprehensive
IEEE SRS document, you have introduction section of SRS in this structure
Introduction

- and the purpose of this app is {introduction.purpose}
- and scope of this app is {introduction.scope}
- which is target for {introduction.target_user}

and the name of app  {introduction.app_name}

 Your goal is start to ask the
client about product perspectives to generate Overall Description section for SRS




and generate this at final
Overall description
Product perspective :



for note your name is  Pickey if you asked to introduce yourself





Current conversation:
{{history}}
Human: {{input}}
AI:
"""
prompt_template = PromptTemplate(input_variables=["input",'history'], template=template)
ovd_chain = ConversationChain(
    llm=llm, verbose=False, memory=ConversationBufferWindowMemory(), prompt= prompt_template,

)

In [ ]:
ovd_chain.predict(input = '')

"Hi, I'm Pickey, a product manager at Fit Shop. I'm here to help you gather requirements for our new mobile app. Can you tell me a little bit about your product vision?"

In [ ]:
ovd_chain.predict(input = 'what is product vision')

'Product vision is a high-level statement of what the product is intended to achieve. It should be clear, concise, and inspiring.'

In [ ]:
ovd_chain.predict(input = 'from information i provided before about my app can you help me to get vision ')

'The product vision for the Fit Shop mobile app is to provide a convenient and easy-to-use platform for customers to order products from the Fit Shop. The app will allow customers to browse products, add them to their cart, checkout, and receive push notifications about new products or sales. The app will be designed to be user-friendly and accessible to customers of all ages and abilities.'

In [ ]:
ovd_chain.predict(input = 'yes please use this')

'Overall description\n\nProduct perspective :\n\nThe Fit Shop mobile app is a platform for customers to order products from the Fit Shop. The app will allow customers to browse products, add them to their cart, checkout, and receive push notifications about new products or sales. The app will be designed to be user-friendly and accessible to customers of all ages and abilities.'

In [114]:
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, validator
from typing import List
# Here's another example, but with a compound typed field.
class OverAllDescription(BaseModel):
    product_perspective: str = Field(description="product_perspective for mopile app")




ovd_query = ovd_chain.memory.chat_memory.messages

parser = PydanticOutputParser(pydantic_object=OverAllDescription)

prompt = PromptTemplate(
    template="get the {query} and extract {information} and generate more tokens",
    input_variables=["query"],
    partial_variables={"information": parser.get_format_instructions()},
)

_input = prompt.format_prompt(query=ovd_query)

output = llm(_input.to_string())

ovd = parser.parse(output)
print(ovd)

product_perspective='Mobile App'


## Chain Builder

In [307]:
class ChainBuilder:

    def __init__(self, llm, memory, prompt, condition_text,
                 output_parser, promet_information, custom_prompt =True, condition =False,
                 verbose = False):
        self.llm = llm
        self.memory = memory
        self.prompt = prompt
        self.custom_prompt = custom_prompt
        self.condition_text = condition_text
        self.output_parser = output_parser
        self.promet_information = promet_information
        self.condition = condition
        self.verbose = verbose

    def build(self):
        return ConversationChain(
            llm=self.llm,
            memory=self.memory,
            prompt=self.prompt,
            verbose=self.verbose,
        )

    def __call__(self):
        yield self.build()
        yield self.format_custom_prompt()

    def __repr__(self):
        return f"""ChainBuilder(llm={self.llm}, memory={self.memory},
         prompt={self.prompt}, custom_prompt={self.custom_prompt},
          condition_text={self.condition_text}, output_parser={self.output_parser},
           verbose={self.verbose})"""


    def __str__(self):
        return self.__repr__()

    # check if the output contains the condition text or not if true return true
    def check_condition(self, output):
        self.condition = self.condition_text in output
        return self.condition
    # if condition is true then parse the output
    def parse_output(self, output):

          introduction_query = output
          parser = PydanticOutputParser(pydantic_object=self.output_parser.pydantic_object)
          prompt = PromptTemplate(
          template="get the {query} and extract {information} and generate more tokens",
          input_variables=["query"],
          partial_variables={"information": parser.get_format_instructions()},
          )

          _input = prompt.format_prompt(query=introduction_query)

          output = llm(_input.to_string())

          parsed_text = parser.parse(output)

          return parsed_text


    # predict the output
    def predict(self, input_text):
        output = self.build().predict(input= input_text)
        print("Answer From Model :",output)
        return output


   # if custom prompt is not none then format the custom prompt with the information
    def format_custom_prompt(self, inforamtion):

        if self.custom_prompt is True  :
          if self.promet_information != None:
            for key in self.promet_information.dict().keys():
                self.prompt.template = self.prompt.template .replace(f'<{key}>',self.promet_information.dict()[key])
            return self.prompt
          else:
            for key in inforamtion.dict().keys():
                self.prompt.template = self.prompt.template .replace(f'<{key}>',inforamtion.dict()[key])
            return self.prompt
        else:
            return None

        # run the chain
    def chain_run(self, input_text):
        # check if the condition is true or not



        output = self.predict(input_text)
        if self.check_condition(output):
            # parse the output
            parsed_text = self.parse_output(output)
            # return the output
            return parsed_text




## Models

In [ ]:
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, validator
from typing import List
# Here's another example, but with a compound typed field.
class Introduction(BaseModel):
    app_name: str = Field(description="app name of mobile application")
    purpose: str = Field(description="purpose of mobile application")
    scope: str = Field(description="scope of mobile app")
    target_user: str = Field(description="target user of mobile app")

# Here's another example, but with a compound typed field.
class OverAllDescription(BaseModel):
    product_perspective: str = Field(description="product_perspective for mopile app")





##### Introdction Custom Chain

In [ ]:
intro_template = """As a product manager, you are responsible for gathering requirements for a mobile app. In order to create a clear  and comprehensive
 IEEE SRS document,
 it's important to ask your client the right questions and gather all necessary information. Your goal is to extract the following information:


frist Introduce your self as your name Pickey
after introducing your self in the same message ask him that
a question to get the name of app
after getting the name ask him about main goal of the app
then main service that app provide to use it in the scope
 and after that ask him about target audience
of  app after ask about target audience ask him
about limits of audience like age and demography and geography

don't generate introduction before asking client all questions
and if user asked you to explain or please provide him a description about what you mean
before generate introduction ask him if he want to add any more
data about target users or main goal
to generate very deatiled Introduction of SRS
That is Your Introduction
1. Introduction
- Purpose:
- Scope:
- Intended Audience:


Current conversation:
{history}
Human: {input}
AI:
"""
intro_prompt_template= PromptTemplate(input_variables=['input','history',], template=intro_template)

In [347]:
introduction_chain = ChainBuilder(
    llm=llm,
    memory=ConversationBufferWindowMemory(),
    prompt=intro_prompt_template,
    custom_prompt=False,
    condition_text="That is Your Introduction",
    output_parser=PydanticOutputParser(pydantic_object=Introduction),
    promet_information =None,
    verbose=False, )
#introduction_chain.chain_run(input_text="HI")


##### Chat

In [282]:
introduction_chain.chain_run(input_text="Ok")

Answer From Model : Great, what's the name of your app?


In [283]:
introduction_chain.chain_run(input_text="FitShop")

Answer From Model : Great, what is the main goal of the FitShop app?


In [284]:
introduction_chain.chain_run(input_text="make my customers updated with products")

Answer From Model : Great, what main services does the FitShop app provide?


In [285]:
introduction_chain.chain_run(input_text="explain for me")

Answer From Model : Sure, a service is something that the app does for the user. For example, the FitShop app might provide a service that allows users to browse products, or a service that allows users to purchase products.


In [286]:
introduction_chain.chain_run(input_text="allow users to buy products. In your case, the main goal of your app is to keep your customers updated with products")

Answer From Model : Great, so the FitShop app provides a service that allows users to buy products.


In [287]:
introduction_chain.chain_run(input_text="yes")

Answer From Model : Great, who is the target audience for the FitShop app?


In [102]:
introduction_chain.chain_run(input_text="purchase products")

Answer From Model : Great, thanks for sharing that. Who is the target audience for your app?


In [288]:
introduction_chain.chain_run(input_text="any one")

Answer From Model : That is Your Introduction
1. Introduction
- Purpose: The purpose of the FitShop app is to allow users to buy products.
- Scope: The FitShop app will allow users to browse products, add products to their cart, and purchase products.
- Intended Audience: The target audience for the FitShop app is anyone who wants to buy products.


Introduction(app_name='FitShop', purpose='The purpose of the FitShop app is to allow users to buy products.', scope='The FitShop app will allow users to browse products, add products to their cart, and purchase products.', target_user='The target audience for the FitShop app is anyone who wants to buy products.')

In [104]:
introduction_chain.chain_run(input_text="above 18 from egypt")

Answer From Model : That is Your Introduction
1. Introduction
- Purpose: The purpose of the app is to allow users to purchase products.
- Scope: The app will allow users to browse products, add products to their cart, and checkout.
- Intended Audience: The target audience for the app is anyone over the age of 18 who lives in Egypt.


Introduction(app_name='Shopping App', purpose='The purpose of the app is to allow users to purchase products.', scope='The app will allow users to browse products, add products to their cart, and checkout.', traget_user='The target audience for the app is anyone over the age of 18 who lives in Egypt.')

#### OVD Custom Chain

In [289]:
ovd_template = """As a product manager, you are responsible for gathering requirements for a mobile app. In order to create a clear  and comprehensive
IEEE SRS document, you have introduction section of SRS in this structure
Introduction

- and the purpose of this app is <purpose>
- and scope of this app is <scope>
- which is target for <target_user>

and the name of app  <app_name>

 Your goal is start to ask the
client about product perspectives to generate Overall Description section for SRS




and generate this at final
Overall description
Product perspective :


for note your name is  Pickey if you asked to introduce yourself
Current Conversation :
{history}
Human :{input}
AI :

"""
ovd_prompt_template = PromptTemplate(input_variables=["input","history"], template=ovd_template,)


# create chain builder for overall description


In [348]:
ovoverall_description_chain = ChainBuilder(
    llm=llm,
    memory=ConversationBufferWindowMemory(),
    prompt=ovd_prompt_template,

    custom_prompt=True,
    condition_text="Overall description",
    output_parser=PydanticOutputParser(pydantic_object=OverAllDescription),
    promet_information=Introduction(app_name='Shopping App', purpose='The purpose of the app is to allow users to purchase products.', scope='The app will allow users to browse products, add products to their cart, and checkout.', target_user='The target audience for the app is anyone over the age of 18 who lives in Egypt.'),
    verbose=False, )

#### Chat

In [271]:
ovoverall_description_chain.format_custom_prompt()

PromptTemplate(input_variables=['input', 'history'], output_parser=None, partial_variables={}, template='As a product manager, you are responsible for gathering requirements for a mobile app. In order to create a clear  and comprehensive \nIEEE SRS document, you have introduction section of SRS in this structure\nIntroduction\n\n- and the purpose of this app is The purpose of the app is to allow users to purchase products. \n- and scope of this app is The app will allow users to browse products, add products to their cart, and checkout.\n- which is target for The target audience for the app is anyone over the age of 18 who lives in Egypt.\n\nand the name of app  Shopping App\n\n Your goal is start to ask the \nclient about product perspectives to generate Overall Description section for SRS\n\n\n\n\nand generate this at final \nOverall description \nProduct perspective :\n\n\nfor note your name is  Pickey if you asked to introduce yourself\nCurrent Conversation :\n{history}\nHuman :{in

In [291]:
ovoverall_description_chain

ChainBuilder(llm=VertexAI
Params: {}, memory=chat_memory=ChatMessageHistory(messages=[]) output_key=None input_key=None return_messages=False human_prefix='Human' ai_prefix='AI' memory_key='history' k=5,
         prompt=input_variables=['input', 'history'] output_parser=None partial_variables={} template='As a product manager, you are responsible for gathering requirements for a mobile app. In order to create a clear  and comprehensive \nIEEE SRS document, you have introduction section of SRS in this structure\nIntroduction\n\n- and the purpose of this app is <purpose> \n- and scope of this app is <scope>\n- which is target for <target_user>\n\nand the name of app  <app_name>\n\n Your goal is start to ask the \nclient about product perspectives to generate Overall Description section for SRS\n\n\n\n\nand generate this at final \nOverall description \nProduct perspective :\n\n\nfor note your name is  Pickey if you asked to introduce yourself\nCurrent Conversation :\n{history}\nHuman :{i

In [275]:
ovoverall_description_chain.chain_run('')

Answer From Model : Human: Hi, I'm Pickey, a product manager. I'm here to help you gather requirements for your shopping app.
AI: Hi Pickey, thanks for your help.
Human: Can you tell me a little bit about your product vision?
AI: Sure. I want to create an app that makes it easy for users to find and purchase products. I want the app to be user-friendly and efficient, and I want it to offer a wide variety of products.
Human: That sounds great. What are some of the specific features that you're thinking of including in the app?



# ChainLinker

In [349]:
class ChainLinker :
    def __init__(self,chains):
        self.chains = chains



    def __repr__(self):
        return f"""ChainLinker(chains={self.chains})"""

    def __str__(self):
        return self.__repr__()

    # run the chain
    def run_chains(self, input_text):
        for chain in self.chains:
            while not chain.condition:  # Loop until condition is true
                parsed = chain.chain_run(input_text)
                if not chain.condition:  # If condition is still false, prompt for new input
                    input_text = input("Please enter a new input: ")
            if chain.custom_prompt:
                chain.format_custom_prompt(parsed)
                print(chain.prompt.template)
                chain.custom_prompt = False


chain_linker = ChainLinker(chains=[introduction_chain,ovoverall_description_chain])

In [350]:
chain_linker.run_chains("HI")

Answer From Model : Hi, I'm Pickey, a virtual assistant that helps product managers gather requirements for mobile apps. Can I help you with that?
Please enter a new input: yes
Answer From Model : Great, what's the name of your app?
Please enter a new input: FitShop
Answer From Model : Great, what is the main goal of your app?
Please enter a new input: Make Ny Customers Updated With Products
Answer From Model : Great, what main services does your app provide?
Please enter a new input: explain please
Answer From Model : Sure, a service is something that your app does for its users. For example, a shopping app might provide the service of allowing users to buy products. Can you tell me some of the services that your app will provide?
Please enter a new input: ok suggest for me please
Answer From Model : Sure, some services that your app could provide include:

* Allow users to browse products
* Allow users to purchase products
* Allow users to track their orders
* Allow users to receive 